In [28]:
from music21 import chord, note, stream, clef, meter, converter
import os
from tqdm import tqdm
import pandas as pd

In [2]:
path_to_classic = "/Users/rpbot_mac/Documents/GitHub/MIDI-gen-ai/Ultimate-MIDI-Scraper/data/MIDIdata/classic"

In [ ]:
os.listdir(path_to_classic)

In [7]:
def open_midi(file_path):
    try:
        midi = converter.parse(file_path)
        return midi
    except Exception as e:
        print(f"An error occurred while opening the MIDI file: {e}")

In [14]:
def create_data_set(dest_directory):
    all_chords = []
    for root, dirs, files in os.walk(dest_directory):
        print(dirs)
        for f_i, _file in tqdm(enumerate(files), desc="Loaded: "):
            mid_file = os.path.join(root, _file)
            midi_data = open_midi(mid_file)
            all_chords.append("<song_start>")
            for i, part in enumerate(midi_data.parts):  # type: ignore
                all_chords.append(f"<part_start>")
                for element in part.recurse():
                    # if isinstance(element, meter.TimeSignature):
                    #     all_chords.append(element.ratioString)
                    if isinstance(element, chord.Chord):
                        all_chords.append("<meta>")
                        all_chords.append(
                            f"chord_quarterlength {element.duration.quarterLength.real}"
                        )
                        all_chords.append(f"chord_offset {element.offset}")
                        all_chords.append("<chord_start>")
                        for note_i, n in enumerate(element):
                            all_chords.append(
                                f"note_pitch index_{note_i} value_{n.pitch.midi}"
                            )  # type: ignore
                            all_chords.append(
                                f"note_velocity index_{note_i} value_{n.volume.velocity}"
                            )
                            all_chords.append(
                                f"note_quarterlength index_{note_i} value_{n.duration.quarterLength}"
                            )
                            all_chords.append(
                                f"note_offset index_{note_i} value_{n.offset}"
                            )
                        all_chords.append("<chord_end>")
            all_chords.append("<song_end>")

    return all_chords

In [10]:
folder_path = (
    "/Users/rpbot_mac/Documents/GitHub/MIDI-gen-ai/Ultimate-MIDI-Scraper/data/MIDIdata"
)

In [15]:
chord_set = create_data_set(folder_path)

['pop', 'classic', 'rap_hip_hop', 'rock', 'rnb_soul', 'country']


Loaded: : 0it [00:00, ?it/s]

An error occurred while opening the MIDI file: cannot find a format extensions for: /Users/rpbot_mac/Documents/GitHub/MIDI-gen-ai/Ultimate-MIDI-Scraper/data/MIDIdata/.DS_Store


AttributeError: 'NoneType' object has no attribute 'parts'

In [24]:
my_list = []

df = pd.DataFrame(my_list, columns=["Giga_MIDI_Language"])

# Write DataFrame to Parquet
df.to_parquet("large_deconstructed_midi.parquet")

In [ ]:
# Read parquet file
df = pd.read_parquet("classical_deconstructed_midi.parquet")

# Display the DataFrame
classic_list = df["classical"].values.tolist()

In [73]:
def filter_data2(inpute_list):
    # filtered_list = inpute_list
    indexes_to_del = []
    song_start_flag = False
    genre_flag = False
    artist_flag = False

    song_start_index = 0
    genre_index = 0
    artist_name_index = 0
    for i, string_ in enumerate(inpute_list):
        if string_ == "<song_start>":
            song_start_index = i
            song_start_flag = True
            continue

        elif (
            string_ == "<classic>"
            and song_start_flag == True
            and i == song_start_index + 1
        ):
            genre_index = i
            genre_flag = True

            continue

        elif (
            string_.startswith("<artist ") == True
            and song_start_flag == True
            and genre_flag == True
            and i == genre_index + 1
        ):
            artist_name_index = i
            artist_flag = True
            continue

        elif (
            string_ == "<song_end>"
            and song_start_flag == True
            and artist_flag == True
            and genre_flag == True
            and i == artist_name_index + 1
        ):
            indexes_to_del.append(song_start_index)
            indexes_to_del.append(genre_index)
            indexes_to_del.append(artist_name_index)
            indexes_to_del.append(i)

        elif (
            string_ == "<part_start>"
            and song_start_flag == True
            and artist_flag == True
            and genre_flag == True
            and i == artist_name_index + 1
        ):
            song_start_flag = False
            genre_flag = False
            artist_flag = False
    return indexes_to_del

In [79]:
indexes_to_filter = filter_data2(classic_list)

In [80]:
filtered_language = []
for index, value in enumerate(classic_list):
    if index not in indexes_to_filter:
        filtered_language.append(value)

In [89]:
filtered_df = pd.DataFrame(filtered_language, columns=["classic"])
filtered_df.to_parquet("classic_deconstructed_midi.parquet")

In [90]:
df = pd.read_parquet("classic_deconstructed_midi.parquet")

# Display the DataFrame
df

,classic
0,<song_start>
1,<classic>
2,<artist Abdelmoine Alfa>
3,<part_start>
4,<chord_meta>
...,...
3236572,note_velocity value_127
3236573,note_quarterlength value_1.0
3236574,note_offset value_0.0
3236575,<chord_end>
